# LDA Try on Real Data (incomplete)

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

## Sequence Data

In [13]:
annotate_data_file_path='annotate_peaks_GRCh38_hg38/annotate_peaks_data_GRCh38_hg38.tsv'
raw_data=pd.read_csv(annotate_data_file_path,sep='\t',low_memory=False)

In [14]:
peak_id_cols=[col for col in raw_data.columns if 'PeakID' in col]
raw_data.rename(columns={peak_id_cols[0]:'PeakID'},inplace=True)
print(f'Raw-Data Shape = {raw_data.shape}')
raw_data.head()

Raw-Data Shape = (60100, 322)


,PeakID,Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,...,"CRE(bZIP)/Promoter/Homer Distance From Peak(sequence,strand,conservation)","Isl1(Homeobox)/Neuron-Isl1-ChIP-Seq(GSE31456)/Homer Distance From Peak(sequence,strand,conservation)","LHX9(Homeobox)/Hct116-LHX9.V5-ChIP-Seq(GSE116822)/Homer Distance From Peak(sequence,strand,conservation)","Tcf12(bHLH)/GM12878-Tcf12-ChIP-Seq(GSE32465)/Homer Distance From Peak(sequence,strand,conservation)","EHF(ETS)/LoVo-EHF-ChIP-Seq(GSE49402)/Homer Distance From Peak(sequence,strand,conservation)","MYB(HTH)/ERMYB-Myb-ChIPSeq(GSE22095)/Homer Distance From Peak(sequence,strand,conservation)","Foxa3(Forkhead)/Liver-Foxa3-ChIP-Seq(GSE77670)/Homer Distance From Peak(sequence,strand,conservation)","Ascl1(bHLH)/NeuralTubes-Ascl1-ChIP-Seq(GSE55840)/Homer Distance From Peak(sequence,strand,conservation)","LEF1(HMG)/H1-LEF1-ChIP-Seq(GSE64758)/Homer Distance From Peak(sequence,strand,conservation)","JunB(bZIP)/DendriticCells-Junb-ChIP-Seq(GSE36099)/Homer Distance From Peak(sequence,strand,conservation)"
0,seq_40800,chr4,20984016,20984165,+,0,NaN,NaN,NaN,-81,...,0,0,0,0,0,0,0,1,0,0
1,seq_10314,chr11,76444716,76444865,+,0,NaN,NaN,NaN,-228,...,0,0,0,0,0,0,0,0,0,0
2,seq_18866,chr15,67558636,67558785,+,0,NaN,NaN,NaN,9616,...,0,0,0,0,0,0,0,0,0,0
3,seq_45348,chr5,134512248,134512397,+,0,NaN,NaN,NaN,5770,...,0,0,2,0,0,0,0,0,0,0
4,seq_2616,chr1,92804876,92805025,+,0,NaN,NaN,NaN,-12540,...,0,0,0,0,0,1,0,0,0,0


In [15]:
motif_cols=[]
for col in raw_data.columns:
  if '/Homer' in col:
    motif_cols.append(col)
print(f'Total No. of Motif = {len(motif_cols)}')

Total No. of Motif = 301


## Know Motif Data

In [16]:
motif_file_path='output_GRCh38_hg38/knownResults.txt'
motif_data=pd.read_csv(motif_file_path,sep='\t')
print(f'Motif-Data Shape = {motif_data.shape}')
motif_data.head()

Motif-Data Shape = (440, 9)


,Motif Name,Consensus,P-value,Log P-value,q-value (Benjamini),# of Target Sequences with Motif(of 60108),% of Target Sequences with Motif,# of Background Sequences with Motif(of 59450),% of Background Sequences with Motif
0,CTCF(Zf)/CD4+-CTCF-ChIP-Seq(Barski_et_al.)/Homer,AYAGTGCCMYCTRGTGGCCA,0.0,-24440.0,0.0,12286.0,20.44%,713.4,1.20%
1,BORIS(Zf)/K562-CTCFL-ChIP-Seq(GSE32465)/Homer,CNNBRGCGCCCCCTGSTGGC,0.0,-18580.0,0.0,14267.0,23.74%,1764.5,2.98%
2,"OCT4-SOX2-TCF-NANOG(POU,Homeobox,HMG)/mES-Oct4...",ATTTGCATAACAATG,0.0,-3513.0,0.0,3076.0,5.12%,417.5,0.71%
3,Fosl2(bZIP)/3T3L1-Fosl2-ChIP-Seq(GSE56872)/Homer,NATGASTCABNN,0.0,-2532.0,0.0,4418.0,7.35%,1220.3,2.06%
4,Fra2(bZIP)/Striatum-Fra2-ChIP-Seq(GSE43429)/Homer,GGATGACTCATC,0.0,-2500.0,0.0,5365.0,8.93%,1731.3,2.92%


#### Identify Duplicate Motifs (based on their names)

In [17]:
#duplicate motifs
motif_data[motif_data['Motif Name'].duplicated(keep=False)]

,Motif Name,Consensus,P-value,Log P-value,q-value (Benjamini),# of Target Sequences with Motif(of 60108),% of Target Sequences with Motif,# of Background Sequences with Motif(of 59450),% of Background Sequences with Motif
268,RORgt(NR)/EL4-RORgt.Flag-ChIP-Seq(GSE56019)/Homer,AAYTAGGTCA,0.00001,-11.67,0.0,536.0,0.89%,436.5,0.74%
269,RORgt(NR)/EL4-RORgt.Flag-ChIP-Seq(GSE56019)/Homer,AAYTAGGTCA,0.00001,-11.67,0.0,536.0,0.89%,436.5,0.74%


## Preprocessing

### Rename Motif Columns

In [18]:
data=raw_data.copy()
motif_dict={}
for idx,row in motif_data.iterrows():
    motif_dict[row['Motif Name']+' Distance From Peak(sequence,strand,conservation)']='motif_'+str(idx+1)
    
data.rename(columns=motif_dict,inplace=True)
new_motif_cols=[col for col in data.columns if 'motif_' in col]
data[['PeakID']+new_motif_cols].head()

,PeakID,motif_100,motif_101,motif_102,motif_103,motif_104,motif_105,motif_106,motif_107,motif_108,...,motif_91,motif_92,motif_93,motif_94,motif_95,motif_96,motif_97,motif_98,motif_99,motif_9
0,seq_40800,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,seq_10314,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,seq_18866,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,seq_45348,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
4,seq_2616,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


### Create Documents

In [19]:
motif_string=[]
for idx,row in tqdm(data.iterrows()):
    motif_list=[]
    for motif in new_motif_cols:
        motif_list.extend(row[motif]*[motif])
    _motifs=','.join(motif_list)
    motif_string.append(_motifs)
    if len(motif_list)==0:
        print(idx,row['PeakID'])
data['motif_string']=motif_string

19032it [00:29, 711.00it/s]

18840 seq_57232


24067it [00:36, 794.97it/s]

23892 seq_20216


29839it [00:44, 749.84it/s]

...

46997it [01:06, 743.49it/s]

46832 seq_8465


47389it [01:07, 792.65it/s]

47219 seq_46451


60100it [01:24, 710.34it/s]


## Save Data

In [20]:
X=data[['PeakID','Distance to TSS','motif_string']]
X.to_csv('DATA.csv',index=False)
X

,PeakID,Distance to TSS,motif_string
0,seq_40800,-81,"motif_149,motif_162,motif_166,motif_166,motif_..."
1,seq_10314,-228,motif_144
2,seq_18866,9616,"motif_104,motif_126,motif_126,motif_147,motif_..."
3,seq_45348,5770,"motif_120,motif_121,motif_128,motif_128,motif_..."
4,seq_2616,-12540,"motif_100,motif_103,motif_126,motif_126,motif_..."
...,...,...,...
60095,seq_5507,-127293,"motif_137,motif_137,motif_13,motif_174,motif_1..."
60096,seq_11070,37721,"motif_132,motif_132,motif_132,motif_142,motif_..."
60097,seq_17945,88589,"motif_161,motif_162,motif_162,motif_174,motif_..."
60098,seq_29137,93216,"motif_104,motif_105,motif_117,motif_117,motif_..."
